In [23]:
# first run installations and data download
! pip install -U llama-index llama-index-vector-stores-milvus pymilvus llama-index-llms-openai llama-index-readers-file
! mkdir -p './storage/commits'


zsh:1: command not found: pip


In [24]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from pymilvus import connections
from llama_index.vector_stores.milvus import MilvusVectorStore

In [25]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/commits"
    )
    commit_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [26]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [27]:
if not index_loaded:
    # load data
    commit_docs = SimpleDirectoryReader(
        input_files=["./0001-adding-a-better-calculator.patch"]
    ).load_data()

    # build index
    vector_store_commit = MilvusVectorStore(dim=1536, collection_name="commit", overwrite=True)
    
    storage_context_commit = StorageContext.from_defaults(vector_store=vector_store_commit)
    commit_index = VectorStoreIndex.from_documents(commit_docs, storage_context=storage_context_commit)


    # persist index
    commit_index.storage_context.persist(persist_dir="./storage/commit")


In [28]:
commit_engine = commit_index.as_query_engine(similarity_top_k=3)

In [29]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=commit_engine,
        metadata=ToolMetadata(
            name="commit_summary",
            description=(
                "Provides summary information about a commit patch.  "
                # "Need to manually run 'git format-patch -1 to generate the patch file."
            ),
        ),
    ),
]

In [30]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [31]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [37]:
agent1_response = agent.chat("What changes were made in this patch?")
print(str(agent1_response))

Thought: I need to use a tool to help me answer the question.
Action: commit_summary
Action Input: {'input': 'this patch'}
Observation: Answer: introduces a better calculator agent with functions for multiplication, addition, and squaring integers. It also includes a tool to interact with the OpenAI model "gpt-4" and demonstrates step-by-step calculations for various mathematical operations like addition, multiplication, and squaring.
Thought: I can answer without using any more tools.
Answer: The changes made in this patch include introducing a better calculator agent with functions for multiplication, addition, and squaring integers. It also includes a tool to interact with the OpenAI model "gpt-4" and demonstrates step-by-step calculations for various mathematical operations like addition, multiplication, and squaring.
The changes made in this patch include introducing a better calculator agent with functions for multiplication, addition, and squaring integers. It also includes a to

In [38]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

In [39]:
def set_title(commit_body) -> str:
    """Takes in a commit body and returns a recommended blog title"""
    #submit commit_body to OpenAI's GPT-3.5 API
    openai = OpenAI()
    # generate a title from the commit body using open ai
    response = openai.completion_to_prompt(commit_body)
    return response


title_tool = FunctionTool.from_defaults(fn=set_title)

In [40]:
llm = OpenAI(model="gpt-3.5-turbo")
#  llm = OpenAI(model="gpt-4")
agent2 = ReActAgent.from_tools([title_tool], llm=llm, verbose=True)

In [41]:
response_gen = agent2.stream_chat(str(agent1_response))
response_gen.print_response_stream()

ValidationError: 1 validation error for Task
input
  str type expected (type=type_error.str)